# Rust Proxy and LB Server

This is a work in progress starting out as a simple Proxy Server/Load Balancer written in Rust. 

- Features
  - Container friendly development and deployment
  - Simple code for the proxy server
  - Load Balancer is round robin algo
    - TODO: Add least used connections algo

# `docker-dev.ps1` Summary

This PowerShell script is used to set up and run the `Docker Compose` environment for the Rust proxy server project. It reads environment variables from a `.env` file and exports them before running `Docker Compose` commands.

## Script Details

- **Filepath**: `{root-workspace-dir}/docker-dev.ps1`

### Steps

1. **Define the location of the `.env` file**:
   ```powershell
   $envFile = "./.env"
   ```
1. Check if the `.env` file exists:
    ```powershell
    if (!(Test-Path $envFile)) {
        Write-Error ".env file not found!"
        exit 1
    }
    ```
1. Read each line in the `.env` file (except comments):
    ```powershell
    Get-Content $envFile | Where-Object { $_ -notmatch '^#' -and $_ } | ForEach-Object {
        # Split the line into key and value
        $key, $value = $_.Split('=', 2)

        # Export the variable using Set-Item
        Set-Item Env:$key $value
    }
    ```

1. Run Docker Compose commands with exported variables:
   ```powershell
    docker compose build
    docker compose up -d
    ```
    > Note DO NOT use `docker-compose` (note the dash or hyphen) syntax seen in some older blog posts.  The `docker-compose` was a stand alone application and has been deprecated.  It is now embedded as a subcommand within Docker.

### Usage

To use this script, run it in a PowerShell terminal:
```powershell
./docker-dev.ps1
```
### Summary
- The script checks for the existence of a .env file.
- It reads the environment variables from the .env file, ignoring comments.
- It exports the environment variables to the current session.
- It runs docker compose build to build the Docker images.
- It runs docker compose up -d to start the Docker Compose services in detached mode.

This script simplifies the process of setting up and running the Docker Compose environment by automatically exporting the necessary environment variables.



# `main.rs` Summary

This file contains the entry point for the TCP proxy server application written in Rust using the `tokio` crate. It reads configuration from environment variables, parses backend addresses, and starts the proxy server.

## Main Function

### `main`

```rust
#[tokio::main]
async fn main() -> Result<(), Box<dyn std::error::Error>>
```

- **Description**: The entry point of the application. It reads the listen address and backend addresses from environment variables, parses them, and starts the proxy server.
- **Returns**: A `Result` indicating success or failure.

## Implementation Details

- The `main` function uses the `tokio::main` attribute to run the function as an asynchronous main function.
- It reads the listen address and backend addresses from environment variables using the `env::var` function.
- The backend addresses are parsed into `SocketAddr` using the `parse` method.
- If any address fails to parse, an error message is printed using `eprintln!.
- The `start_proxy` function from the `rust_proxy_lb_adaptive` library is called with the parsed listen address and backend addresses to start the proxy server.

## Example Usage

To run the proxy server, ensure that the required environment variables are set and then execute the application:

```bash
export RUST_LOG=info
export BACKEND1_ADDR=backend1:8081
export BACKEND2_ADDR=backend2:8082
export BACKEND3_ADDR=backend3:8083

cargo run
```

The environment variables should be set in a `.env` file or directly in the command line. The proxy server will listen on the specified address and forward traffic to the backend servers in a round-robin manner.


# `lib.rs` Summary

This file contains the core logic for a TCP proxy server written in Rust using the `tokio` crate. The proxy server listens on a specified address and forwards traffic to a pool of backend servers using a round-robin algorithm.

## Functions

### `handle_client`

```rust
pub async fn handle_client(
    mut inbound: TcpStream,
    proxy_addr: SocketAddr,
) -> Result<(), Box<dyn std::error::Error>>
```

- **Description**: Handles the communication between the inbound client connection and the outbound connection to the backend server.
- **Parameters**:
  - `inbound`: The incoming `TcpStream` from the client.
  - `proxy_addr`: The `SocketAddr` of the backend server to forward the traffic to.
- **Returns**: A `Result` indicating success or failure.

### `start_proxy`

```rust
pub async fn start_proxy(
    listen_addr: SocketAddr,
    backends: Vec<SocketAddr>,
) -> Result<(), Box<dyn std::error::Error>>
```

- **Description**: Starts the proxy server, listens for incoming connections, and forwards them to the backend servers using a round-robin algorithm.
- **Parameters**:
  - `listen_addr`: The `SocketAddr` to listen for incoming connections.
  - `backends`: A vector of `SocketAddr` representing the backend servers.
- **Returns**: A `Result` indicating success or failure.


## Implementation Details

- The `handle_client` function splits the inbound and outbound connections into read and write halves and uses `tokio::io::copy` to forward data between them.
- The `start_proxy` function creates a `VecDeque` of backend addresses and uses a `Mutex` to ensure thread-safe access.
- The proxy server listens for incoming connections on the specified address and spawns a new task for each connection.
- The backend addresses are rotated in a round-robin manner to distribute the load evenly across the backend servers.

## Example Usage

To use the proxy server, call the `start_proxy` function with the desired listen address and a list of backend addresses:


```rust
let listen_addr = "0.0.0.0:8068".parse::<SocketAddr>()?;
let backend_addrs = vec![
    "127.0.0.1:8081".parse::<SocketAddr>()?,
    "127.0.0.1:8082".parse::<SocketAddr>()?,
    "127.0.0.1:8083".parse::<SocketAddr>()?,
];

start_proxy(listen_addr, backend_addrs).await?;
```
